In [ ]:
import time
import dask
import rhg_compute_tools.kubernetes as rhgk
import rhg_compute_tools.utils as rhgu
import fsspec
import xarray as xr

In [2]:
client, cluster = rhgk.get_micro_cluster()

In [ ]:
print('https://compute.impactlab.org' + cluster.dashboard_link)

In [3]:
N_WORKERS = 5
cluster.scale(N_WORKERS)

In [ ]:
# wait until the workers come online to do anything else
while len(client.ncores()) < N_WORKERS:
    time.sleep(5)

In [ ]:
try:
    fs = fsspec.filesystem(
        'gs',
        timeout=120,
        cache_timeout=120,
        requests_timeout=120,
        read_timeout=120,
        conn_timeout=120,
    )
    ds = xr.open_zarr(fs.get_mapper('gs://downscaled-288ec5ac/stage/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/tasmax/gn/v20220125010335.zarr'))
    res = ds.isel(lat=0, lon=0).compute().mean()
    print(res)
except Exception:
    client.close()
    cluster.close()
    raise

In [5]:
client.close()
cluster.close()